# Matlab Beispiele aus der VL

## ModSim1

### Numerische Simulation – Folie 12

In [ ]:
function energienetz
    ...
    %-- ELektrolyse
    i_RE=0 ;i_2E=0;i_SE=0;U_5=V_ref*10;mp_E=0;i_E=0;
    %-- PV Anlage
    i_PV=0;i_6=0;i_RPV=0;
    %-- Anfangswerte
    y0 = [i_R;U_1;i_B;Q_B;i_BZ;i_S;P_1;m_BZ;U_2;U_K;i_2;i_RE ;i_2E;...
    i_SE;U_5;mp_E;i_E;i_V;i_PV;i_6;i_RPV];
    %-- Matrix M, Zeitspanne, Options
    M = zeros(21);M(4,4)=1;M(7,7)=1;M(10,10)=1;
    tspan = [0,24*3600];
    options = odeset( 'Mass' ,M, 'RelTol' ,1.0e-8, 'AbsTol' ,1.0e-6, 'Stats','on' );
    [t,y] = ode15s(@dgl,tspan,y0,options ,Z1,Z2,Z3);
    %-- Darstellung
    plot(t/3600,y, '-o' );
    legend( 'i_R','U_1(Kennlinie)','i_B','Q_B(Ladung)',....
    ...
end

function dy = dgl(t,y)
    %-- rechte Seite des DAE Systems M y'=f(t,y)
    i_R = y(1); U_1 = y(2); ...
    ...
    dy(1) =i_R*U_3-U_2*i_BZ; %-- Spannungswandler
    dy(2) = i_S-i_R; %-- Knoten U2
    ....
    %--
end

### Algorithmus – Folie 50

In [ ]:
for k=1:n-1 % durchlaufe alle Diagonalelemente
	faktor = 1/A(k,k)
	for i=k+1:n % durchlaufe Spalten unterhalb a_kk
		A(i,k) = A(i,k)*faktor % merke dir die Zahl mit der
	end % Zeile k multipliziert werden muss
	for i=k+1:n % alle Zeilen unterhalb
		for j=k+1:n % alle Spalten rechts
			A(i,j) = A(i,j) - A(i,k)*A(k,j) % Zeile i - Zeile k
		end
	end
end

### Algorithmus – Folie 56

In [ ]:
% a) Ly = b (von oben nach unten)
for j = 1:n
	y(j) = b(j)
	for i = j+1:n % schaufele Spalte j
		b(i) = b(i) - A(i,j)*y(j) % nach rechts
	end
end

% b) Ux = y (von unten nach oben)
for j = n:-1:1
	x(j) = y(j)/A(j,j)
	for i = 1:j-1 % schaufele Spalte j
		y(i) = y(i) - A(i,j)*x(j) % nach rechts
	end
end

- Aufruf in MATLAB: – Folie 57

In [ ]:
A=[4,-1,0; -1,4,-1; 8,-1,4]
x=A\b

### MATLAB-Beispiel – Folie 64

In [ ]:
A=[4,-1,0;8,-1,4;-1,4,-1]; b=[2;4;6];
[L,U,P]=lu(A) % LU-Zerlegung aufrufen

- Folie 65

In [ ]:
y=L\(P*b); % Rücksubstitution
x=U\y
x=A\b % oder direkt

### Wie hoch ist der Rechenaufwand des GA bzw. der LU-Zerlegung? – Folie 67

1. LU-Zerlegung:

In [ ]:
for k=1:n-1
	....
	for i=k+1:n
		for j=k+1:n
		...
		end
	end
end

2. Rücksubstitution:

In [ ]:
for j = 1:n
	for i = j+1:n
	...
	end
end
for j = n:-1:1
	for i = 1:j-1
	...
	end
end

### Eigenschaften der Fixpunktiteration zur Bestimmung von Nullstellen einer Funktion f (x) – Folie 80

Die Fixpunktiteration ist sehr einfach zu implementieren (auch für mehrdimensionale Probleme):

In [ ]:
while abs(g(x)-x) > tol begin
	x = g(x)
en

### Implementierung des Straßenprofils: – Folie 103

In [ ]:
function strassenkante
    load strasse.dat
    global xi yi p %-- die messwerte werden in der Fkt. fcn benötigt
    xi = strasse(:,1); yi = strasse(:,2);
nx = length(xi)
plot(xi,yi); hold on
%-- Startwerte und Testplot
a = min(yi)
b = max(yi)-a
diff = abs(yi(2:nx)-yi(1:nx-1)); [diffmax,ii] = max(diff);
c = xi(ii)
p = 5*(xi(end)-xi(1))/nx
params = [a;b;c;p];
yi1 = fit_fcn(xi,params);
plot(xi,yi,xi,yi1,'r')
hold on

### Newtonverfahren - Folie 104

In [ ]:
%-- newtonverfahren
tol = 1.0e-6;
x0 = params(1:3); %-- startwert des Newtonverfahrens
xres = newton(@fcn,x0,tol)
xres = [xres;p];
yi2 = fit_fcn(xi,xres);
plot(xi,yi2,'Linewidth',3)
function y = fit_fcn(x,params)
a = params(1); b = params(2); c = params(3); p = params(4);
y = a + b./(exp((x-c)/p)+1);

- Folie 105

In [ ]:
function x = newton(fcn,x0,tol)
    global xi yi p %-- die messwerte werden in der Fkt. fcn benötigt
    h = [2*tol;2*tol];
    x = x0;
    nit =0;
    while (norm(h) > tol) && (nit < 20)
        nit = nit+1;
        J=jac(x,fcn);
        b=-1.*fcn(x);
        h=J\b;
        x = x + h;
        params = [x;p];
    end

- Folie 106

In [ ]:
function f=fcn(x)
    global xi yi p %-- die messwerte werden in der Fkt. fcn benötigt
    a = x(1); b = x(2); c = x(3);
    params = [x;p];
    f0 = fit_fcn(xi,params);
    expp = exp( (xi-c)/p );
    f(1,1) = sum(f0-yi);
    f(2,1) = sum( (f0-yi)./(expp + 1) );
    f(3,1) = sum( (f0-yi)*b/p.* expp./(expp+1).^2 );

- Folie 107

In [ ]:
function J=jac(x,f)
    n = length(x);
    f0 = f(x);
    for i = 1:n
        h = sqrt(eps)*max(1.0e-8,abs(x(i)));
        x1 = x; x1(i)=x1(i)+h;
        J(:,i) = (f(x1)-f0)/h;
    end

### Wie sehen Interpolationspolynome aus? – Folie 118

In [ ]:
x=linspace(-4,4,1000);
y=exp(-x.^2);
for i=1:9
	x1=linspace(-4,4,i);
	y1=exp(-x1.^2);
	p=polyfit(x1,y1,i-1); % Interpolationsploynom
			 % vom Grade i-1
	y2=polyval(p,x); % Auswertung des Polynoms
	subplot(3,3,i);
	plot(x,y,x1,y1,'o',x,y2);
end

### Wie sehen Ausgleichspolynome aus? – Folie 126

In [ ]:
x=linspace(0,5,1000);
y=exp(-x);
x1=linspace(0,5,11);
y1=exp(-x1);
for i=1:4
	p=polyfit(x1,y1,i-1);
	y2=polyval(p,x);
	subplot(2,2,i);
	plot(x,y,x1,y1,'o',x,y2);
end

- 2) MATLAB Befehle – Folie 130

In [ ]:
load bonn.dat;
plot(bonn(:,1),bonn(:,2),'x');

- Ausgleichspolynom 3. Grades – Folie 131

In [ ]:
pp=polyfit(bonn(:,1),bonn(:,2),3)

liefert $p(x) = 268.56 + 3.41 x + 0.00616 x^2 + 5.9 \cdot 10^{-7} x^3$.

- Graphische Darstellung

In [ ]:
y=polyval(pp,bonn(:,1));
plot(bonn(:,1),y,bonn(:,1),bonn(:,2),'x');

### MATLAB – Folie 152

In [ ]:
function f=rohr(x)
    f=sqrt(4-x.^2);
% Hauptprogramm (Trapezregel)
format long;
a=1.0; b=2.0;
n=100; h=(b-a)/n;
sum=rohr(a)+rohr(b);
x=a;
for i=1:n-1
	x=x+h;
	sum=sum+2.0*rohr(x);
end
sum=h*sum/2 	% ist das Ergebnis

- Folie 153

Vergleich: MATLAB-Routine `quad` (=adaptives Simpsonverfahren)

In [ ]:
q=quad(@rohr,1.,2.)

### Anwendungsbeispiel zu ode45 – Folie 174

Lösung des AWP

$$y' = -2y, y(0) = 1$$

und Vergleich mit der exakten Lösung $y(t) = e^{-2t}$

In [ ]:
y0 = 1;
tspan = [0, 3];
[t,y] = ode45(@dgl, tspan, y0);
plot(t,y,'x',t,exp(-2*t));
function dy = dgl(t,y)
dy = -2*y;

- Folie 175

Höhere Genauigkeit = kleinere Zeitschritte

In [ ]:
...
options = odeset('RelTol',1.0e-6,'AbsTol',1.0e-8);
[t,y] = ode45(@dgl, tspan, y0, options);
...

### Beispiel – Folie 177

Lösung des AWP

$$y'' = -y, y(0) = 0, y'(0) = 1$$

und Vergleich mit der exakten Lösung $y(t) = \sin(t)$

In [ ]:
y0 = [0 1];
tspan = [0, 2*pi];
[t,y] = ode45(@dgl, tspan, y0);
plot(t,y(:,1),'x',t,sin(t));
function dz = dgl(t,z)
    dz(1,1) = z(2);
    dz(2,1) = -z(1);

### Folie 182

In [ ]:
function masse_1
    g = 9.81; x0 = 0; u0 = 10; z0 = 0; v0 = 10;
    dt = 0.01; k=1; % Zaehler, Zeitschritt
    x(k) = x0; z(k) = z0; t(k) = 0;
    while z(k) >= 0
        k = k+1;
        t(k) = t(k-1) + dt;
        x(k) = u0*t(k) + x0;
        z(k) = -0.5*g*t(k)^2 + v0*t(k) + z0;
    end
    plot(x,z)

- Folie 183

In [ ]:
function masse_2
    % animierte Darstellung
    g = 9.81; x0 =0; u0 = 10; z0 = 0; v0 = 10;
    dt = 0.01; k=1; % Zaehler, Zeitschritt
    x(k) = x0; z(k) = z0; t(k) = 0;
    axis equal; axis([0 21 0 6]);
    r = 0.1;
    kreis = rectangle('Position',[x(k)-r z(k)-r 2*r 2*r],...
            'Curvature',[1 1],'FaceColor','r');
    while z(k) >= 0
        k = k+1;
        t(k) = t(k-1) + dt; x(k) = u0*t(k) + x0;
        z(k) = -0.5*g*t(k)^2 + v0*t(k) + z0;
        set(kreis,'Position',[x(k)-r z(k)-r 2*r 2*r]);
        drawnow
    end

- Folie 186

In [ ]:
function masse_3
    %--
    x0 =0; u0 = 10; z0 = 0; v0 = 10;
    tspan = [0 2];
    y0 = [x0 u0 z0 v0];
    options = odeset('RelTol',1.0e-6);
    [t,y] = ode45(@dgl, tspan, y0, options);
    plot(y(:,1),y(:,3))

function dy = dgl(t,y)
    g = 9.81;
    dy(1,1) = y(2);
    dy(2,1) = 0;
    dy(3,1) = y(4);
    dy(4,1) = -g;

In [ ]:
% mit Animation...
[t,y] = ode45(@dgl, tspan, y0, options);
r = 0.1;
axis equal
axis([0 21 0 6]);
line(y(:,1),y(:,3))
kreis = rectangle('Position',[x0-r z0-r 2*r 2*r],...
		'Curvature',[1 1],'FaceColor','r');
for i=1:length(t)
	set(kreis,'Position',[y(i,1)-r y(i,3)-r 2*r 2*r]);
	drawnow
	pause(0.1)
end

- Folie 188

In [ ]:
% Hauptprogramm
options = odeset('RelTol',1.0e-6,'Events',@events);
[t,y] = ode45(@dgl, tspan, y0, options);
....
function [value,isterminal,direction]=events(t,y)
    %-- Eventfunktion, stoppt Integration bei z=0
    isterminal = 1; % stoppe Simulation
    direction = -1; % nur bei fallender event-Funktion
    value= y(3);

- Folie 202

In [ ]:
function masse_abhang
    global g m
    g = 9.81; m = 2;
    y0 = [0 0 0 0 g*m/2];
    tspan = [0 20];
    M = eye(5); M(5,5)=0;
    options = odeset('Mass',M);
    [t,z] = ode15s(@dgl, tspan, y0,options);
    plot(z(:,1),z(:,3))

function dz = dgl(t,z)
    global g m
    dz(1,1) = z(2);
    dz(2,1) = z(5)/m;
    dz(3,1) = z(4);
    dz(4,1) = g - z(5)/m;
    dz(5,1) = g - 2*z(5)/m;

## ModSim2

### Die Potenzmethode – Folie 45

In [ ]:
%%file power_iteration.m
function x = power_iteration (A, numIterations)
	x = rand ( size (A, 2), 1 ); % random starting vector
	for i = 1:numIterations
		x = A*x ; % calculate matrix- vector product
		x = x/norm( x ) ; % normalize
	end

- Folie 46

In [ ]:
A=[0.7 0.4 ; 0.3 0.6 ] ; % rental bike example
power_iteration (A, 1) % 1 iteration
power_iteration (A,10) % 10 iterations
power_iteration (A, 100) % 100 iterations

### Das QR–Verfahren – Folie 50

In [ ]:
%%file qr_iteration.m
function A = qr_iteration (A, numIterations )
	for i =1: numIterations
		[Q, R] = qr (A) ; % A = Q*R
		A = R*Q;
	end

### Beispiel: Ein eindimensionaler Stab – Folie 52

In [ ]:
[eigenmodes, D] = eig (M\K) );
eigenfreqs = sqrt ( diag (D) );

### Beispiel: Verrauschtes Signal – Folie 88

In [ ]:
% create a function consisting of two sinus waves
T=1.5; % end time
dt=1e-3; % sampling time
t = 0: dt : T-dt ; % Time vector
S = 0.7* sin (2* pi *50* t ) + sin (2* pi *120* t ) ;
% corrupt signal with noise of variance 4
X = S + 2*randn ( size ( t ) ) ;
plot ( t ,X) ;
xlabel ( 'time [s]' )
ylabel ( 'Amplitude' )

### Beispiel: Verrauschtes Signal – Folie 90

In [ ]:
% calculate (real) spectrum
Y = fft (X) ;
N = length (Y) ;
f = (0:N-1)/T;
a = 2* real (Y)/N;
b =-2*imag(Y)/N;

%plot the results
figure ;
stem( f (1:N/2) ,[ a (1:N/2); b (1:N/2 ) ] ' ) ;
legend ( ' ak ' , 'bk ' ) ;
xlabel ( ' Frequency [Hz] ' )
ylabel ( ' Amplitude ' )

### Zum Testen des Abtast–Theorems: – Folie 95

In [ ]:
%%file an_annoying_script.m
fs = 8000;	% sample frequency
T=0.25;		% duration for each sound
for f =100:100:12000
	% if f > fs/2 ,f will be captured incorrectly
	disp ( [ ' f = ' , num2str ( f ) ,' Hz, 2* f / fs = ' , num2str(2* f / fs ) ] )
	delta_t = 1/ fs ;
	t=0: delta_t :T;
	y=sin (2* pi * f *t ) ;
	soundsc (y , fs )
	pause (1.05*T)
end

### Beispiel – Folie 128

In [ ]:
% boundary values
bvleft = 0;	bvright = 0;
% define 1. order ODE
ode = @(x , y)	[ y (2); -exp (y ( 1 ) ) ] ;
% define the BVs as a function handle
bcs = @(ya ,yb)	[ ya(1)-bvleft ; yb(1)-bvright ] ;
% define initial guess ( negative parabola w/ zeros at 0 and 1)
x = linspace (0, 1, 5);
winit = @(x) [20* x.*(1-x ) ; 20*(1-2*x ) ] ;
solinit = bvpinit (x, winit ) ;
% solve the BVP
sol = bvp4c (ode, bcs, solinit) ;

In [ ]:
sol
xdense = linspace (0, 1, 100);
ydense = deval ( sol, xdense );

### Mehrdimensionale Funktionen – Folie 146

In [ ]:
[X, Y] = meshgrid( -1:0.05:1 ,-1:0.05:1);
surf(X, Y, X.^2 + Y. ^ 2 );
xlabel( ' x ' );
ylabel( ' y ' );
zlabel( ' z ' );
title( ' z=f (x , y)=x^2 + y^2 ' )

### Mehrdimensionale Funktionen – Folie 148

In [ ]:
t=linspace (0 ,20 ,100);
plot3( cos ( t ), sin( t ), t ) ;
xlabel( ' x ' ) ;
ylabel( ' y ' ) ;
zlabel( ' z ' ) ;

### Linienmethode in Matlab - Hauptroutine – Folie 214

In [ ]:
% constant velocity
u= 15;
% initial condition
q0= @(x) exp(-(x-25000).^2*1e-8);
% left boundary condition
qa= 0;
% time domain
t0= 0;		t1= 6000;
% space domain
x0= 0;		x1= 100000;
% discretized space domain
N= 100;
x= linspace (x0 , x1 , N) ;
dx= (x1-x0 )/(N-1);

%% method of lines
A= constructMatrix (N) ;
b= zeros (N-1 ,1);
b(1)= -qa ;

% solve using ode45
y0= q0(x (2: end ) ) ' ;
ode= @( t , y) -(u/dx)*A*y + b ;
[ tmol ,ymol]= ode45(ode , [ t0 , t1 ] , y0 ) ;

- Hauptroutine

In [ ]:
%%file constructMatrix.m
function A = constructMatrix (N)
%% construct matrix stencil for
% backwards differences as a
% sparse matrix
nnz = 2*N-3;
i=zeros (nnz , 1 ) ;
j=zeros (nnz , 1 ) ;
v=zeros (nnz , 1 ) ;
i (1)=1;
j (1)=1;
v(1)=1;
idx = 2;
for k=1:N-2
	i ( [ idx  idx +1])=[k+1 k+1];
	j ( [ idx  idx +1])=[k k+1];
	v ( [ idx  idx +1])=[ -1  1 ] ;
	idx = idx + 2;
end
A=sparse ( i , j , v ,N-1,N-1);
end

- Visualisierung

In [ ]:
% plot initial conitions
figure ;
hold on ;
hexact = plot (x , q0(x ) , . . .
	' LineWidth ' , 1 . 5 , . . .
	' DisplayName ' , ' exact solution ' ) ;
hmol = plot (x , [ qa ymol ( 1 , : ) ] , . . .
	' LineWidth ' , 1 . 5 , . . .
	' DisplayName ' , 'Method of Lines ' ) ;
xlabel( ' x ' ) ;
ylabel( ' y ' ) ;
xlim( [ x0 ,x1 ] )
ylim( [ 0 , 1 ] )
legend( ' toggle ' )

### Matlab Implementierung – Folie 228

In [ ]:
%%file heat2d.m
% collect all global variables in a structure for better readability
global params

% parameters
params.k = 237;	% heat conductivity [W/mK]
params.c = 900;	% heat capacity [ J/kgK]
params.p = 2700;	% density [ kg/m^3]
params.P = 5;	% power of the heat source [W]

params.sT = 20;	% surrounding Temperature [C]
params.lambda = . . .	% heat diffusivitiy [m^2/s ]
           params.k/(params.c*params.p ) ;

params.x1 = 0.07;	% width the plate [m]
params.width = 0.025;	% thickness of the plate [m]
params.t1 = 40;	% end time [ s ]

params.nx = 40;	% # discretization points in each direction
params.dx = . . .	% x-resolution [m]
            params.x1/(params.nx-1);
            
%% solve the system
% initial conditions (BCs are applied in odeFun)
T0vec = params.sT*ones ( params.nx^2 ,1);
sol = ode15s (@odeFun, [0 , params.t1 ] , T0vec ) ;

%% animate the solution
[X, Y]=meshgrid ( linspace (0 , params . x1 , params . nx ) ) ;
t= linspace (0 , params . t1 , 200);
Tvec = deval ( sol , t ) ; %evaluate solution at the given time steps
for i =1:length ( t )
	T = reshape (Tvec ( : , i ) , params.nx , params.nx ) ;
	surf (X,Y,T) ;		% plot Temperature as surface
	axis ([0 , params.x1 , . . . 	% x-axis limits
	          0 , params.x1 , . . . 	% y-axis limits
	          18  32 , . . .		% z-axis limits
	          18 32 ] ) ;		% color axis limits
	colorbar ;
	title ( [ ' t = ' , num2str ( t ( i )) , ' s ' ] )
	zlabel ( 'T [C] ' ) ;
	drawnow ;
end

In [ ]:
%%file odeFun.m
function dTvec = odeFun( t , Tvec)
    % compute the ode for every point on the  plate using Matlabs conv2
    global params ; % make sure the parameters are available
    T = reshape (Tvec, params.nx, params.nx ) ; % reshape to 2D array
    % pad T to obtain zero inflow boundary conditions
    T=[T( 1 , : ) ;T;T(end , : ) ] ;
    T=[T( : , 1 ) T T( : , end ) ] ;
    %convolute T with the 5-point-stencil
    stencil = ( params.lambda/params.dx ^ 2 ) * . . .
        [0 , 1 , 0;
        1 , -4, 1;
        0 ,   1 ,0 ] ;
    dT = conv2 (T, stencil , 'same ' ) ;
    dT = dT(2: end -1,2:end-1); % remove padding
    dT = dT + heatsource ( t ) ; % add heat source
    dTvec = reshape (dT, params.nx ^2 ,1); % reshape to vector

In [ ]:
%%file heatSource.m
function f = heatsource ( t )
    % apply the heat sources at (0.02 ,0.05) and (0.02 ,0.035)
    global params ; % make sure the parameters are available
    % get index of closest points to heat sources
    X = (0: params.nx)*params.dx ; 
    [~ , ix1 ] = min( abs (X-0.02));
    [~ , ix2 ] = min( abs (X-0.05));
    [~ ,i y] = min( abs (X-0.035));
    f = zeros ( params.nx ) ; % 2D (n , n)-array of zeros
    % switch heat source off after 20 seconds
    Tsource = params.P/(params.c*params.p*params.width*params.dx^2)
    if ( t<20 )
        f ( iy , ix1 ) = f ( iy , ix1 ) + Tsource ;
        f ( iy , ix2 ) = f ( iy , ix2 ) + Tsource ;
    end

### Beispiel: Elastische Schnur mit Last – Folie 260

In [ ]:
%%file fem_elastic_string.m
xa = 0.4; 			% left boundary of load interval
xb = 0 .6;			% right boundary of load interval
q0 = -1;				% constant load in load interval
k = 1;				% ” Diffusion ” coefficient
L = 1;				% length of the string
alph = 0;				% left Dirichlet boundary value
bet = 0;				% right Dirichlet boundary value
q = @(x) q0*(x>=xa ).*( x<=xb ) ; 	% distributed load depends on x

N = 100;				% number of finite elements
h = L/N;				% stepsize
x = linspace (0 ,L ,N+1) ';		% spatial discrete points

%% assemble the stiffness matrix
nnz = 3*(N-1) -2;
i = zeros (nnz , 1 ) ;
j = zeros (nnz , 1 ) ;
v = zeros (nnz , 1 ) ;
% first row
i (1:2)=[1 , 1 ] ' ;
j (1:2)=[1 , 2 ] ' ;
v (1:2)=[2 , -1] ';
idx = 3;
% rows 2 to N-2
for
	i i =2:N-2
	i ( idx : idx+2)=[ ii ,     ii , ii ] ' ;
	j ( idx : idx+2)=[ ii -1, ii , ii +1] ';
	v( idx : idx+2)=[    -1, 2 ,    -1] ';
	idx = idx + 3;
end
% last row
i ( idx : idx+1)=[N-1, N-1] ';
j ( idx : idx+1)=[N-2, N-1] ';
v( idx : idx+1)=[ -1, 2 ] ' ;
A = sparse ( i , j , v ,N-1,N-1);

% right- hand-side
b = (h^2/k)*q(x (2: end -1));
b(1) = b(1) + alph ;
b(N-1) = b(N-1) + bet ;

%% solve the linear system
w = A\b ;
W = [ alph ; w; bet ] ; % apply BCs

%% plot the results
plot (x ,W, ' LineWidth ' , 1 , . . .
	  ' DisplayName ' , . . .
	  'FEM solut ion ' )
legend ( ' toggle ' )